# Table of Contents<a name="toc"></a>
* [Implement PKCS\#7 padding](#prob9)
* [Implement CBC mode](#prob10)
* [An ECB/CBC detection oracle](#prob11)
* [Byte-at-a-time ECB decryption (Simple)](#prob12)

In [34]:
%%capture
!pip install pycryptodome
from Crypto.Cipher import AES
import base64
import binascii
import numpy as np
import random
from typing import Tuple, List
from enum import Enum, auto

# Implement PKCS\#7 padding<a name="prob9"></a>

In [2]:
def pkcs7(data: bytes, *, block_size: int = 16) -> bytes:
    padding = block_size - (len(data) % block_size)
    return data + bytes([padding for x in range(padding)])

In [17]:
pkcs7(b"YELLOW SUBMARINE", block_size=20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

# Implement CBC mode<a name="prob10"></a>

<img src="img/CBC_decryption.svg">
<img src="img/CBC_encryption.svg">
So, if I understand AES CBC decryption correctly, the way it works is provided an initialization vector, a ciphertext block is deciphered and then XOR'd with the initialization vector to get the resulting plaintext. The next ciphertext block is decrypted  and XOR'd with the previous ciphertext block. CBC encryption works the same, only going the other way.

In [5]:
def xor_encrypt(cipher: bytes, block: bytes) -> bytes:
    cipher_npa = np.frombuffer(cipher, dtype=np.uint8)
    block_npa = np.frombuffer(block, dtype=np.uint8)
    return np.bitwise_xor(cipher_npa, block_npa).tobytes()
xor_decrypt = xor_encrypt

In [9]:
def aes_cbc_decipher(ciphertext: bytes, key: bytes, iv: bytes) -> bytes:
    decipher = AES.new(key, AES.MODE_ECB)
    plaintext = b""
    previous_block = iv
    while len(ciphertext) > 0:
        segment = ciphertext[:len(iv)]
        ciphertext = ciphertext[len(iv):]
        plaintext += xor_decrypt(decipher.decrypt(segment), previous_block)
        previous_block = segment
    return plaintext

In [26]:
def aes_cbc_encipher(plaintext: bytes, key: bytes, iv: bytes) -> bytes:
    encipher = AES.new(key, AES.MODE_ECB)
    ciphertext = b""
    previous_block = iv
    while len(plaintext) > 0:
        segment = plaintext[:len(iv)]
        plaintext = plaintext[len(iv):]
        encoded_block = encipher.encrypt(xor_encrypt(segment, previous_block))
        ciphertext += encoded_block
        previous_block = encoded_block
    return ciphertext

In [30]:
key = b"YELLOW SUBMARINE"
iv = b"\x00" * 16

with open("set2/10.txt") as fd:
    data = fd.read()
data = data.replace("\n", "")

ciphertext = base64.b64decode(data)
plaintext = aes_cbc_decipher(ciphertext, key, iv)

# check to see if encryptor works
encoded_text = aes_cbc_encipher(plaintext, key, iv)
assert(encoded_text == ciphertext)

print(plaintext.decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


[Return to top](#toc)

# An ECB/CBC detection oracle<a name="prob11"></a>

In [36]:
class AESMode(Enum):
    ECB = auto()
    CBC = auto()
    
    def __str__(self):
        return f"{self.name.lower()}"

    
def generate_key(num_bytes: int=16) -> bytes:
    return bytes([random.randint(0,255) for x in range(num_bytes)])
generate_iv = generate_key


def aes_dice_roll() -> AESMode:
    dice_roll = random.randint(0, 1)
    if dice_roll == 0:
        return AESMode.ECB
    else:
        return AESMode.CBC

In [37]:
def encryption_oracle(plaintext: bytes, block_size: int=128) -> Tuple[List[str], bytes]:
    # pre- and appending random 5-10 bytes to plaintext
    plaintext = bytes([random.randint(0, 255) for x in range(random.randint(5, 10))]) + plaintext
    plaintext += bytes([random.randint(0, 255) for x in range(random.randint(5, 10))])
    
    plaintext = pkcs7(plaintext, block_size=block_size)
    
    ciphertext = b""
    ciphermode = []
    while len(plaintext) > 0:
        segment = plaintext[:block_size]
        plaintext = plaintext[block_size:]
        roll = aes_dice_roll()
        
        if roll == AESMode.ECB:
            ciphermode.append(str(roll))
            key = generate_key()
            encipher = AES.new(key, AES.MODE_ECB)
            ciphertext += encipher.encrypt(segment)
        elif roll == AESMode.CBC:
            ciphermode.append(str(roll))
            key = generate_key()
            iv = generate_iv()
            ciphertext += aes_cbc_encipher
            
    return (ciphermode, ciphertext)

[Return to top](#toc)

# Byte-at-a-time ECB decryption (Simple)<a name="prob12"></a>

[Return to top](#toc)